# 0. Importer les bibliotheques necessaires

In [1]:
import pandas as pd
import numpy as np
import pysubgroup as ps
from sklearn.preprocessing import Imputer

# 1. Importer le jeu de donnees

In [2]:
data = pd.read_csv('restaurant_data_kaggles.csv')
data.head(5)

,restaurant_name,review_number,food_type,ranking,overallRating,wifi,livraison,average_price,lat,lng,district
0,le Neuvième Art,626 avis,Française Européenne Végétariens bienvenus,Nº 1 sur 3 058 Restaurants à Lyon,5.0,yes,yes,85 € -145 €,45.76849365234375,4.85646390914917,69006 Lyon
1,Aromatic,601 avis,Française Européenne Végétariens bienvenus,Nº 2 sur 3 058 Restaurants à Lyon,5.0,yes,no,18 € -41 €,45.77438735961914,4.830961227416992,69004 Lyon
2,Le Boeuf d'Argent,482 avis,Française Européenne Végétariens bienvenus,Nº 3 sur 3 058 Restaurants à Lyon,4.5,no,no,no informations,45.76266098022461,4.826900005340576,69005 Lyon
3,Le Comptoir des Cousins,205 avis,Française Européenne Sud-américaine,Nº 4 sur 3 058 Restaurants à Lyon,5.0,yes,no,13 € -28 €,45.76849365234375,4.85646390914917,69006 Lyon
4,Le Book-Lard,204 avis,Française Brasserie gastronomique Végétarien...,Nº 5 sur 3 058 Restaurants à Lyon,5.0,yes,no,13 € -40 €,45.76708984375,4.845279216766357,69006 Lyon


## 1.1. Consulter les colonnes dans dataframe

In [3]:
data.columns

Index(['restaurant_name ', 'review_number ', ' food_type ', ' ranking ',
       ' overallRating  ', ' wifi ', ' livraison ', ' average_price ', ' lat ',
       ' lng ', ' district '],
      dtype='object')

# 2. Pre-traiter le jeu de donnees

## 2.1. Supprimer les "espace" dans le nom des colonnes

In [4]:
data.columns = data.columns.str.replace(' ', '')

## 2.2. Eliminer des colonnes "ranking" et "average_price"

In [5]:
data = data.drop(columns = ['ranking', 'average_price'])

## 2.3. Supprimer les lignes contenant la valeur 'no information' dans quelques colonnes

In [6]:
cols = ['review_number', 'food_type', 'overallRating', 'lat', 'lng', 'district']

for c in cols:
    data = data[~data[c].str.contains('no')]
data

,restaurant_name,review_number,food_type,overallRating,wifi,livraison,lat,lng,district
0,le Neuvième Art,626 avis,Française Européenne Végétariens bienvenus,5.0,yes,yes,45.76849365234375,4.85646390914917,69006 Lyon
1,Aromatic,601 avis,Française Européenne Végétariens bienvenus,5.0,yes,no,45.77438735961914,4.830961227416992,69004 Lyon
2,Le Boeuf d'Argent,482 avis,Française Européenne Végétariens bienvenus,4.5,no,no,45.76266098022461,4.826900005340576,69005 Lyon
3,Le Comptoir des Cousins,205 avis,Française Européenne Sud-américaine,5.0,yes,no,45.76849365234375,4.85646390914917,69006 Lyon
5,L'Archange,946 avis,Française Européenne,4.5,no,no,45.76845932006836,4.827991008758545,69001 Lyon
...,...,...,...,...,...,...,...,...,...
2795,Le pop rock,49 avis,Française,1.5,no,no,45.75033950805664,4.8261260986328125,69002 Lyon
2797,Burger King,45 avis,Américaine,1.5,no,no,45.71208953857422,4.9671502113342285,69800 Saint-Priest Lyon
2798,Les Chandelles,861 avis,Française,2.0,no,no,45.76337432861328,4.827271938323975,69005 Lyon
2799,La Carretta,47 avis,Pizza,1.5,no,no,45.76569366455078,4.827927112579346,69005 Lyon


## 2.4. Extrait les type d'aliments dans la colonne "food_type"

Implementer la fonction `split()` permettant extraire le type dans un `String`

In [7]:
def split(food_type):
    if(food_type==''):
        return[]
    list_food_types = []
    for l in food_type:
        if l.isupper():
            if len(list_food_types)!=0:
                list_food_types[-1] = list_food_types[-1][:-2]
            list_food_types.append(l)
        elif len(list_food_types)>0:
            list_food_types[-1]+=l
    for i in list_food_types:
        if i=="Barbecu":
            list_food_types.append("Barbecue/Grillades")
            list_food_types.remove("Grillades")
            list_food_types.remove("Barbecu")
        if i=='Moye':
            list_food_types.append("Moyen_Orient")
            list_food_types.remove("Moye")
            list_food_types.remove("Orient")
        if i=='Europe de ':
            list_food_types.append("Europe de l'Est")
            list_food_types.remove('Europe de ')
            list_food_types.remove("Est")
        if i=='Asiatique  cambodgienne':
            list_food_types.append("Asiatique")
            list_food_types.append('Cambodgienne')
            list_food_types.remove('Asiatique  cambodgienne')
        if i=='Vietnamienne  cambodgienne':
            list_food_types.append("Vietnamienne")
            list_food_types.append('Cambodgienne')
            list_food_types.remove('Vietnamienne  cambodgienne')
    return list_food_types

split("Française  Internationale Barbecue/Grillades")

['Française', 'International', 'Barbecue/Grillades']

Marcher la fonction `split()` pour chaque ligne dans la colonne `food_type` pour obtenir une liste de type d'aliments

In [8]:
food_type = [split(f) for f in data['food_type']]
list_food_type = []
for i in food_type:
    for j in i:
        if j not in list_food_type:
            list_food_type.append(j)
len(list_food_type)

78

Creer un dataframe de `list_food_type`

In [9]:
d = []
for r in data['food_type']:
    d.append([])
    f = split(r)
    for food in list_food_type:
        d[-1].append(int(food in f))
df = pd.DataFrame(d)
df.columns = list_food_type
df

,Française,Européenne,Végétariens bienvenus,Sud-américaine,Méditerranéenne,Internationale,Barbecue/Grillades,Bar à vins,Moderne,Diner,...,Belge,Roumaine,Tunisienne,Suisse,Albanaise,Canadienne,Amérique centrale,Arabe,Vénézuélienne,Chilienne
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1753,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1754,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1755,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1756,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Ajouter les nouvelle colonnes au jeu de donnees

In [10]:
data = data.reset_index(drop=True)
data[df.columns] = df
data

,restaurant_name,review_number,food_type,overallRating,wifi,livraison,lat,lng,district,Française,...,Belge,Roumaine,Tunisienne,Suisse,Albanaise,Canadienne,Amérique centrale,Arabe,Vénézuélienne,Chilienne
0,le Neuvième Art,626 avis,Française Européenne Végétariens bienvenus,5.0,yes,yes,45.76849365234375,4.85646390914917,69006 Lyon,1,...,0,0,0,0,0,0,0,0,0,0
1,Aromatic,601 avis,Française Européenne Végétariens bienvenus,5.0,yes,no,45.77438735961914,4.830961227416992,69004 Lyon,1,...,0,0,0,0,0,0,0,0,0,0
2,Le Boeuf d'Argent,482 avis,Française Européenne Végétariens bienvenus,4.5,no,no,45.76266098022461,4.826900005340576,69005 Lyon,1,...,0,0,0,0,0,0,0,0,0,0
3,Le Comptoir des Cousins,205 avis,Française Européenne Sud-américaine,5.0,yes,no,45.76849365234375,4.85646390914917,69006 Lyon,1,...,0,0,0,0,0,0,0,0,0,0
4,L'Archange,946 avis,Française Européenne,4.5,no,no,45.76845932006836,4.827991008758545,69001 Lyon,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1753,Le pop rock,49 avis,Française,1.5,no,no,45.75033950805664,4.8261260986328125,69002 Lyon,1,...,0,0,0,0,0,0,0,0,0,0
1754,Burger King,45 avis,Américaine,1.5,no,no,45.71208953857422,4.9671502113342285,69800 Saint-Priest Lyon,0,...,0,0,0,0,0,0,0,0,0,0
1755,Les Chandelles,861 avis,Française,2.0,no,no,45.76337432861328,4.827271938323975,69005 Lyon,1,...,0,0,0,0,0,0,0,0,0,0
1756,La Carretta,47 avis,Pizza,1.5,no,no,45.76569366455078,4.827927112579346,69005 Lyon,0,...,0,0,0,0,0,0,0,0,0,0


Supprimer la colonne "food_type"

In [11]:
data = data.drop(columns='food_type')

## 2.5. Traiter la colonne "review_number" 

Supprimmer le String `avis` dans l'entier de la colonne et changer le type de donnees de `String` a `float`

In [12]:
data['review_number'].replace(regex=True,inplace=True,to_replace=r' avis ',value=r'')
data['review_number'] = pd.to_numeric(data['review_number'], errors='coerce')
data['review_number'].isna().sum()

11

Supprimmer les ligne contenant la valeur `np.nan`

In [13]:
data = data.dropna()
data.isna().sum()

restaurant_name      0
review_number        0
overallRating        0
wifi                 0
livraison            0
                    ..
Canadienne           0
Amérique centrale    0
Arabe                0
Vénézuélienne        0
Chilienne            0
Length: 86, dtype: int64

## 2.6. Traiter la colonne "overallRating" 

Changer le type de la colonne de `String` a `Float` et verifier s'elle contient des valeurs `np.nan`

In [14]:
data['overallRating'] = pd.to_numeric(data['overallRating'], errors='coerce')
data['overallRating'].isna().sum()

0

## 2.7. Traiter les colonnes "wifi" et "livraison"

Changer la valeur `yes` a `1.0`et `no` a `0` dans ces colonnes

In [15]:
data['wifi'].replace(inplace=True,to_replace='yes',value=1.0)
data['wifi'].replace(inplace=True,to_replace='no',value=0.0)
data['livraison'].replace(inplace=True,to_replace='yes',value=1.0)
data['livraison'].replace(inplace=True,to_replace='no',value=0.0)
data

,restaurant_name,review_number,overallRating,wifi,livraison,lat,lng,district,Française,Européenne,...,Belge,Roumaine,Tunisienne,Suisse,Albanaise,Canadienne,Amérique centrale,Arabe,Vénézuélienne,Chilienne
0,le Neuvième Art,626.0,5.0,1.0,1.0,45.76849365234375,4.85646390914917,69006 Lyon,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Aromatic,601.0,5.0,1.0,0.0,45.77438735961914,4.830961227416992,69004 Lyon,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Le Boeuf d'Argent,482.0,4.5,0.0,0.0,45.76266098022461,4.826900005340576,69005 Lyon,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Le Comptoir des Cousins,205.0,5.0,1.0,0.0,45.76849365234375,4.85646390914917,69006 Lyon,1,1,...,0,0,0,0,0,0,0,0,0,0
4,L'Archange,946.0,4.5,0.0,0.0,45.76845932006836,4.827991008758545,69001 Lyon,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1753,Le pop rock,49.0,1.5,0.0,0.0,45.75033950805664,4.8261260986328125,69002 Lyon,1,0,...,0,0,0,0,0,0,0,0,0,0
1754,Burger King,45.0,1.5,0.0,0.0,45.71208953857422,4.9671502113342285,69800 Saint-Priest Lyon,0,0,...,0,0,0,0,0,0,0,0,0,0
1755,Les Chandelles,861.0,2.0,0.0,0.0,45.76337432861328,4.827271938323975,69005 Lyon,1,0,...,0,0,0,0,0,0,0,0,0,0
1756,La Carretta,47.0,1.5,0.0,0.0,45.76569366455078,4.827927112579346,69005 Lyon,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2.8. Consulter le types des colonnes

In [16]:
data.dtypes

restaurant_name       object
review_number        float64
overallRating        float64
wifi                 float64
livraison            float64
                      ...   
Canadienne             int64
Amérique centrale      int64
Arabe                  int64
Vénézuélienne          int64
Chilienne              int64
Length: 86, dtype: object

# 3. Sauvegarder le nettoye jeu de donnees dans la ficher "cleaned_data.csv"

In [17]:
data.to_csv(path_or_buf='cleaned_data.csv', index=False)